In [1]:
import random
import math
import gym
import numpy as np
import PIL
from PIL import Image
import matplotlib
import matplotlib.cm as cm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

In [2]:
class model(nn.Module):
    def __init__(self):
        super(model,self).__init__()
        self.hidden_size=512
        self.rnn= nn.RNN(input_size=4, hidden_size=512,num_layers=2,batch_first=True)
        self.fc = nn.Linear(512, 2)
        
    def init_hidden(self,batch_size):
        return (torch.zeros(2,batch_size, self.hidden_size))
    
    def forward(self,x,hidden):
        x=x.reshape(x.shape[0],1,4)
        x,h_0=self.rnn(x,hidden)
        return self.fc(x.contiguous().view(x.size(0), -1))
        

In [3]:
env = gym.make('CartPole-v0').unwrapped
env.reset()

array([-0.00646086, -0.00111536,  0.00539625,  0.04000238])

In [4]:
policy=model()
target_net=model()
target_net.load_state_dict(policy.state_dict())
target_net.eval()
optimizer = optim.RMSprop(policy.parameters())
criterion = F.smooth_l1_loss
memory=10000
store=[[dict()] for i in range(memory)]
gamma=0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200

In [5]:
def addEpisode(ind,prev,curr,reward,act):
    if len(store[ind]) ==0:
        store[ind][0]={'prev':prev,'curr':curr,'reward':reward,'action':act}
    else:
        store[ind].append({'prev':prev,'curr':curr,'reward':reward,'action':act})

In [6]:
def trainNet(total_episodes):
    if total_episodes==0:
        return
    ep=random.randint(0,total_episodes-1)
    if len(store[ep]) < 8:
        return
    else:  
        start=random.randint(1,len(store[ep])-1)
        length=len(store[ep])
        inp=[]
        target=[]
        rew=torch.Tensor(1,length-start)
        actions=torch.Tensor(1,length-start)
        
        for i in range(start,length,1):
            inp.append((store[ep][i]).get('prev'))
            target.append((store[ep][i]).get('curr'))
            rew[0][i-start]=store[ep][i].get('reward')
            actions[0][i-start]=store[ep][i].get('action')
        targets = torch.Tensor(target[0].shape[0],target[0].shape[1],target[0].shape[2])
        torch.cat(target, out=targets)
        ccs=torch.Tensor(inp[0].shape[0],inp[0].shape[1],inp[0].shape[2])
        torch.cat(inp, out=ccs)
        hidden = policy.init_hidden(length-start)
        qvals= target_net(targets,hidden)
        actions=actions.type('torch.LongTensor')
        actions=actions.reshape(length-start,1)
        hidden = policy.init_hidden(length-start)
        inps=policy(ccs,hidden).gather(1,actions)
        p1,p2=qvals.detach().max(1)
        targ = torch.Tensor(1,p1.shape[0])   
        for num in range(start,length,1):
            if num==len(store[ep])-1:
                targ[0][num-start]=rew[0][num-start] 
            else:
                targ[0][num-start]=rew[0][num-start]+gamma*p1[num-start]
        optimizer.zero_grad()
        inps=inps.reshape(1,length-start)
        loss = criterion(inps,targ)
        loss.backward()
        for param in policy.parameters():
            param.grad.data.clamp(-1,1)
        optimizer.step()
            

In [7]:
def trainDRQN(episodes):
    steps_done=0
    avg=0
    for i in range(0,episodes,1):
        if i%100==0 and i!=0:
            print("Episode",i,end=" ")
        env.reset()
        prev,_,_,_=env.step(0)
        prev=torch.from_numpy(prev)
        prev=prev.reshape(1,1,4)
        prev=prev.type('torch.FloatTensor')
        done=False
        steps=0
        rew=0
        while done == False:
            eps_threshold = EPS_END + (EPS_START - EPS_END) * \
            math.exp(-1. * steps_done / EPS_DECAY)
            steps+=1
            hidden = policy.init_hidden(1)
            output=policy(prev,hidden)
            action=(output.argmax()).item()
            rand= random.uniform(0,1)
            if rand < eps_threshold:
                action=random.randint(0,1)
            x = env.step(action)
            sc,reward,done,_=x
            avg=avg+reward
            sc=torch.from_numpy(sc)
            sc=sc.reshape(1,1,4)
            sc=sc.type('torch.FloatTensor')
            rew=rew+reward
            addEpisode(i,prev.unsqueeze(0),sc.unsqueeze(0),reward,action)
            trainNet(i)
            prev=sc
            steps_done+=1
        terminal = torch.zeros(prev.shape[0],prev.shape[1],prev.shape[2])
        addEpisode(i,prev.unsqueeze(0),terminal.unsqueeze(0),-10,action)
        if i%10==0:
            target_net.load_state_dict(policy.state_dict())
        if i%100==0 and i!=0:
            print(" -> Average reward over last 100 episodes " ,avg/100)
            avg=0

In [8]:
trainDRQN(1000)

Episode 100  -> Average reward over last 100 episodes  20.74
Episode 200  -> Average reward over last 100 episodes  57.98
Episode 300  -> Average reward over last 100 episodes  114.74
Episode 400  -> Average reward over last 100 episodes  105.28
Episode 500  -> Average reward over last 100 episodes  125.47
Episode 600  -> Average reward over last 100 episodes  126.84
Episode 700  -> Average reward over last 100 episodes  121.92
Episode 800  -> Average reward over last 100 episodes  136.72
Episode 900  -> Average reward over last 100 episodes  72.21
